# Experimental Music Transformer Version 3 (ver. 0.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install einops
!pip install torch-summary

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')

import os
import pickle
import copy
import statistics
import secrets
import tqdm
import math

from joblib import Parallel, delayed, parallel_config

import torch
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

print('Loading TMIDIX module...')

%cd /content/tegridy-tools/tegridy-tools/

import TMIDIX

print('Loading X Transformer module...')

%cd /content/tegridy-tools/tegridy-tools/X-Transformer

from x_transformer_1_23_2 import *
import random

%cd /content/

print('Creating I/O dirs...')

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

if not os.path.exists('/content/DATA'):
    os.makedirs('/content/DATA')

print('Done!')
print('PyTorch version:', torch.__version__)
print('Enjoy! :)')

# (DOWNLOAD AND UNZIP MIDI DATASET)

In [ ]:
# @title Download and unzip Mono Melodies Piano Violin MIDI Dataset
%cd /content/Dataset
!wget https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Mono-Melodies/Piano-Violin/Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip.001
!wget https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Mono-Melodies/Piano-Violin/Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip.002
!cat Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip* > Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip
!unzip Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip
!rm Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip
%cd /content/

# (LOAD MIDI PROCESSOR)

In [ ]:
#@title TMIDIX MIDI Processor

print('=' * 70)
print('Loading TMIDIX MIDI Processor...')
print('=' * 70)

def TMIDIX_MIDI_Processor(midi_file):

    melody_chords = []

    try:

        fn = os.path.basename(midi_file)

        # Filtering out GIANT4 MIDIs
        file_size = os.path.getsize(midi_file)

        if file_size <= 1000000:

          #=======================================================
          # START PROCESSING

          # Convering MIDI to ms score with MIDI.py module
          score = TMIDIX.midi2single_track_ms_score(open(midi_file, 'rb').read(), recalculate_channels=False)

          # INSTRUMENTS CONVERSION CYCLE
          events_matrix = []
          itrack = 1
          patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

          while itrack < len(score):
              for event in score[itrack]:
                  if event[0] == 'note' or event[0] == 'patch_change':
                      events_matrix.append(event)
              itrack += 1

          events_matrix.sort(key=lambda x: x[1])

          events_matrix1 = []

          for event in events_matrix:
                  if event[0] == 'patch_change':
                        patches[event[2]] = event[3]

                  if event[0] == 'note':
                        event.extend([patches[event[3]]])

                        if events_matrix1:
                            if (event[1] == events_matrix1[-1][1]):
                                if ([event[3], event[4]] != events_matrix1[-1][3:5]):
                                    events_matrix1.append(event)
                            else:
                                events_matrix1.append(event)

                        else:
                            events_matrix1.append(event)

        if len(events_matrix1) > 0:
            if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1]) >= 0:

                #=======================================================
                # PRE-PROCESSING

                # checking number of instruments in a composition
                instruments_list = list(set([y[3] for y in events_matrix1]))

                if len(events_matrix1) > 0:

                    #===================================
                    # ORIGINAL COMPOSITION
                    #===================================

                    # Adjusting timings

                    for e in events_matrix1:
                      e[1] = int(e[1] / 16)
                      e[2] = int(e[2] / 16)

                    # Sorting by patch, pitch, then by start-time

                    events_matrix1.sort(key=lambda x: x[6])
                    events_matrix1.sort(key=lambda x: x[4], reverse=True)
                    events_matrix1.sort(key=lambda x: x[1])

                    #=======================================================
                    # FINAL PROCESSING

                    #=======================================================
                    # MAIN PROCESSING CYCLE
                    #=======================================================

                    pe = events_matrix1[0]

                    notes = []

                    for e in events_matrix1:

                      time = max(0, min(255, (e[1] - pe[1])))
                      dur = max(0, min(255, e[2]))
                      cha = max(0, min(15, e[3]))
                      ptc = max(1, min(127, e[4]))

                      notes.append([time, dur, cha, ptc])

                      pe = e

                    chords = []
                    cho = []

                    for n in notes:

                      if n[2] not in [0, 3]:
                        n[2] = 0

                      if n[0] == 0:
                        chans = list(set([nn[2] for nn in cho]))
                        if (n[2] == 3) and (3 in chans):
                          n[2] = 0

                        cho.append(n)
                      else:
                        if len(cho) > 0:
                          chords.append(cho)

                        cho = []
                        cho.append(n)


                    if len(cho) > 0:
                      chords.append(cho)

                    return chords

    except:
      return None

print('Done!')
print('=' * 70)

# (FILES LIST)

In [ ]:
#@title Save file list
###########

print('=' * 70)
print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"

# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if not filez:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

else:
  print('Randomizing file list...')
  random.shuffle(filez)
  print('Done!')
  print('=' * 70)
  print('Total files:', len(filez))
  print('=' * 70)

# (PROCESS MIDIs)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

melody_chords_f = []

print('Processing MIDI files. Please wait...')
print('=' * 70)

for i in tqdm.tqdm(range(0, len(filez), 16)):

  with parallel_config(backend='threading', n_jobs=4, verbose = 0):

    output = Parallel()(delayed(TMIDIX_MIDI_Processor)(f) for f in filez[i:i+16])

    for o in output:
        if o is not None:
            melody_chords_f.append(o)

print('Done!')
print('=' * 70)

# (SAVE/LOAD PROCESSED MIDIs)

In [ ]:
#@title Save processed MIDIs
TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/DATA/Processed_MIDIs')

In [ ]:
# @title Load processed MIDIs
melody_chords_f = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/DATA/Processed_MIDIs')
print('Done!')

# (PREP INTs)

In [ ]:
# @title Convert processed MIDIs to INTs for training

def split_data_into_chunks(data, chunk_size):
    # Use list comprehension to create chunks of the specified size
    return [data[i:i + chunk_size] for i in range(0, len(data) - len(data) % chunk_size, chunk_size)]

print('=' * 70)

SEQ_LEN = 256

src_list = []
trg_list = []

for m in tqdm.tqdm(melody_chords_f):

  for j in range(-6, 6):

    sdat = []
    tdat = []

    for mmm in m:

      melody_tone = max(1, min(127, mmm[0][3]+j)) % 12

      tones_chord = sorted(list(set([(n[3]+j) % 12 for n in mmm])))

      try:
        chord_id = TMIDIX.ALL_CHORDS.index(tones_chord)
      except:
        chord_id = -1

      if chord_id != -1:
        pchord_id = chord_id

      else:
        pchord_id = TMIDIX.ALL_CHORDS.index([melody_tone])

      sdat.extend([melody_tone])
      tdat.extend([pchord_id])

    if len(sdat) > SEQ_LEN:
      sdat1 = split_data_into_chunks(sdat, SEQ_LEN)

    if len(tdat) > SEQ_LEN:
      tdat1 = split_data_into_chunks(tdat, SEQ_LEN)

    for i in range(len(tdat1)):
      fill_ratio = sum(1 for t in tdat1[i] if t > 11) / len(tdat1[i])

      if fill_ratio >= 0.6:
        src_list.append(sdat1[i])
        trg_list.append(tdat1[i])

print('Done!')
print('=' * 70)
if len(max(src_list, key=len)) == len(min(src_list, key=len)) and len(max(trg_list, key=len)) == len(min(trg_list, key=len)):
  print('All data is good!')
else:
  print('WARNING!!! BAD DATA!!!')
print('=' * 70)

In [ ]:
trg_list[0]

#===========================================================

# (SAVE/LOAD TRAINING INTs)

In [ ]:
# @title Save INTs
TMIDIX.Tegridy_Any_Pickle_File_Writer([src_list, trg_list], '/content/DATA/Training_INTs')

In [ ]:
# @title Load INTs
src_list, trg_list = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/DATA/Training_INTs')
print('Done!')

# (PREP DATA)

In [ ]:
# @title DATA
SEQ_LEN = 256
batch_size = 128

def train_test_split(src, trg, test_size=0.2):
    indices = torch.randperm(len(src)).tolist()
    split = int(test_size * len(src))
    src_train = src[indices[split:]]
    trg_train = trg[indices[split:]]
    src_test = src[indices[:split]]
    trg_test = trg[indices[:split]]
    return src_train, src_test, trg_train, trg_test

# Convert lists to PyTorch tensors
src_in = torch.tensor(src_list).long()
trg_in = torch.tensor(trg_list).long()

# Split the data into train, validation, and test sets
src_train, src_val_test, trg_train, trg_val_test = train_test_split(src_in, trg_in, test_size=0.05)
src_val, src_test, trg_val, trg_test = train_test_split(src_val_test, trg_val_test, test_size=0.3)

class MusicDataset(Dataset):
    def __init__(self, src, trg):
        self.src = src
        self.trg = trg

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        src = self.src[idx].long()
        trg = self.trg[idx].long()
        return src, trg

# Create datasets for each split
train_dataset = MusicDataset(src_train, trg_train)
val_dataset = MusicDataset(src_val, trg_val)
test_dataset = MusicDataset(src_test, trg_test)

# Create data loaders for each split
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [ ]:
train_dataset[0]

# (TRAIN)

In [ ]:
# @title Train model

torch.cuda.empty_cache()

# Initialize the model
model = XTransformer(
    dim = 512,
    enc_num_tokens = 12,
    enc_depth = 8,
    enc_heads = 8,
    enc_max_seq_len = SEQ_LEN,
    enc_dropout = 0.3,
    enc_attn_flash = True,
    dec_num_tokens = 320,
    dec_depth = 8,
    dec_heads = 8,
    dec_max_seq_len = SEQ_LEN,
    dec_dropout = 0.3,
    dec_attn_flash = True,
    cross_attn_tokens_dropout = 0.3
    )

model.cuda()

# Define the optimizer
optimizer = Adam(model.parameters())

# Initialize AMP
scaler = torch.cuda.amp.GradScaler()

NUM_EPOCHS = 100
GRADIENT_ACCUMULATE_EVERY = 1  # Set the number of steps to accumulate gradients

PRINT_STATS_EVERY = 20

train_losses = []
train_accs = []

num_steps = 0

# Training loop with gradient accumulation

for epoch in range(NUM_EPOCHS):  # replace NUM_EPOCHS with the actual number of epochs
    model.train()  # set the model to training mode
    total_loss = 0
    optimizer.zero_grad(set_to_none=True)  # Initialize gradients to zero at the start of the epoch

    for batch_idx, batch in enumerate(tqdm.tqdm(train_loader)):  # iterate over batches of data
        src, tgt = [item.cuda() for item in batch]  # unpack the source and target tensors from the current batch

        src_mask = src.bool().cuda()  # create a mask for the source sequence
        with torch.cuda.amp.autocast():
            loss, acc = model(src, tgt, mask=src_mask)  # forward pass

        # loss = loss / GRADIENT_ACCUMULATE_EVERY  # Normalize the loss by the number of accumulation steps
        scaler.scale(loss).backward()  # Backward pass with gradient scaling

        train_losses.append(loss.mean().item() * GRADIENT_ACCUMULATE_EVERY)
        train_accs.append(acc.mean().item())


        if (batch_idx + 1) % GRADIENT_ACCUMULATE_EVERY == 0:  # Perform optimization step after accumulating gradients
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)  # Reset gradients after optimization step

        total_loss += loss.item() * GRADIENT_ACCUMULATE_EVERY  # Undo the normalization for logging

        if num_steps % PRINT_STATS_EVERY == 0:
            print(f'Training Loss: {total_loss / (batch_idx + 1)}, Accuracy: {acc.item()}')

        num_steps += 1

    # Validation loop
    model.eval()
    with torch.no_grad():
        for i, (src, trg) in enumerate(val_loader):
            src = src.cuda()
            trg = trg.cuda()
            src_mask = src.bool().cuda()  # create a mask for the source sequence
            with torch.cuda.amp.autocast():
              loss, acc = model(src, tgt, mask=src_mask)  # forward pass

            print(f'Validation Loss: {loss.item()}, Accuracy: {acc.item()}')

            if i > 10:
              break

    avg_loss = total_loss / len(train_loader)  # calculate average loss for the epoch
    print(f'Epoch {epoch}: Average Loss: {avg_loss}')


    print('Plotting training loss graph...')

    tr_loss_list = train_losses
    plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
    plt.show()
    plt.close()
    print('Done!')

    print('Plotting training acc graph...')

    tr_loss_list = train_accs
    plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
    plt.show()
    plt.close()
    print('Done!')

# EVAL

In [ ]:
# Validation loop
model.eval()
with torch.no_grad():
    for i, (src, trg) in enumerate(test_loader):
        src = src.cuda()
        trg = trg.cuda()
        src_mask = src.bool().cuda()  # create a mask for the source sequence
        with torch.cuda.amp.autocast():
          loss, acc = model(src, tgt, mask=src_mask)  # forward pass

        print(f'Validation Loss: {loss.item()}, Accuracy: {acc.item()}')

#===========================================================

In [ ]:
# Convering MIDI to ms score with MIDI.py module

midi_file = '/content/tegridy-tools/tegridy-tools/seed-melody.mid'

score = TMIDIX.midi2single_track_ms_score(open(midi_file, 'rb').read(), recalculate_channels=False)

# INSTRUMENTS CONVERSION CYCLE
events_matrix = []
itrack = 1
patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

while itrack < len(score):
    for event in score[itrack]:
        if event[0] == 'note' or event[0] == 'patch_change':
            events_matrix.append(event)
    itrack += 1

events_matrix.sort(key=lambda x: x[1])

events_matrix1 = []

for event in events_matrix:
        if event[0] == 'patch_change':
              patches[event[2]] = event[3]

        if event[0] == 'note':
              event.extend([patches[event[3]]])

              if events_matrix1:
                  if (event[1] == events_matrix1[-1][1]):
                      if ([event[3], event[4]] != events_matrix1[-1][3:5]):
                          events_matrix1.append(event)
                  else:
                      events_matrix1.append(event)

              else:
                  events_matrix1.append(event)

if len(events_matrix1) > 0:
  if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1]) >= 0:

      #=======================================================
      # PRE-PROCESSING

      # checking number of instruments in a composition
      instruments_list = list(set([y[3] for y in events_matrix1]))

      if len(events_matrix1) > 0:

          #===================================
          # ORIGINAL COMPOSITION
          #===================================

          # Adjusting timings

          for e in events_matrix1:
            e[1] = int(e[1] / 16)
            e[2] = int(e[2] / 16)

          # Sorting by patch, pitch, then by start-time

          events_matrix1.sort(key=lambda x: x[6])
          events_matrix1.sort(key=lambda x: x[4], reverse=True)
          events_matrix1.sort(key=lambda x: x[1])

          #=======================================================
          # FINAL PROCESSING

          #=======================================================
          # MAIN PROCESSING CYCLE
          #=======================================================

          pe = events_matrix1[0]

          notes = []

          for e in events_matrix1:

            time = max(0, min(255, (e[1] - pe[1])))
            dur = max(0, min(255, e[2]))
            cha = max(0, min(15, e[3]))
            ptc = max(1, min(127, e[4]))

            notes.append([time, dur, cha, ptc])

            pe = e

mel_pitches = [n[3] for n in notes]
mel_tones = [n[3] % 12 for n in notes]

In [ ]:
# @title Eval the model
dtype = 'float16'
device_type = 'cuda'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

model.eval()

x = torch.tensor(mel_tones, dtype=torch.long, device='cuda')[None, ...]

prime = []

for t in mel_tones[:8]:
  prime.append(TMIDIX.ALL_CHORDS.index([t]))
y = torch.tensor(prime, dtype=torch.long, device='cuda')[None, ...]

#x = torch.tensor([[0]] * 1, dtype=torch.long, device='cuda')

# run generation

with ctx:
    out = model.generate(x, y,
                        seq_len=x.shape[1]-9,
                        temperature=0.85,
                        return_prime=True,
                        verbose=True)

y = out.tolist()

print('=' * 70)
print(y[0])
print('=' * 70)

In [ ]:
for c in y[0]:
  print(TMIDIX.ALL_CHORDS[c])

In [ ]:
mel_tones[:8]

In [ ]:
#@title Test model output

train_data1 = y[0]

#train_data1 = max(melody_chords_f, key = len)

print('Sample INTs', train_data1[:15])

out = train_data1

patches = [0] * 16
patches[3] = 40

if len(out) != 0:

    song = out
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    mel_idx = 0

    for ss in song:

        time += notes[mel_idx][0] * 16
        dur = notes[mel_idx][1] * 16
        pitch = notes[mel_idx][3]
        channel = 3
        song_f.append(['note', time, dur, channel, pitch, vel ])

        chord = TMIDIX.ALL_CHORDS[ss]
        for c in chord:
          pitch = 48+c
          channel = 0
          song_f.append(['note', time, dur, channel, pitch, vel ])

        mel_idx += 1

detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Experimental Music Transformer',
                                                          output_file_name = '/content/Experimental-Music-Transformer-Composition',
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=patches
                                                          )

In [ ]:
torch.save(model.state_dict(), '/content/model.pth')

# (TOKENS EMBEDDINGS)

In [ ]:
# @title Explore model tokens embeddings
tok_emb = model.decoder.net.token_emb.emb.weight.detach().cpu().tolist()

cos_sim = metrics.pairwise_distances(
  tok_emb, metric='cosine'
)
plt.figure(figsize=(7, 7))
plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
plt.xlabel("Position")
plt.ylabel("Position")
plt.tight_layout()
plt.plot()
plt.savefig("/content/Experimental-Music-Transformer-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# Congrats! You did it! :)